In [1]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
#Folder for the dataset
datasetFolder = '/Users/rparundekar/dataspace/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 4#256

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    dataX=pd.read_csv(datasetFolder + datasetXFile)
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    del dataX['id']
    del dataY['id']

    return dataX, dataY

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

Loading datasetX_1.csv & datasetY_1.csv


In [5]:
dataX.head()

,walk_1,walk_2,walk_3,walk_4,walk_5,walk_6,walk_7,walk_8,walk_9,walk_10,...,walk_3542,walk_3543,walk_3544,walk_3545,walk_3546,walk_3547,walk_3548,walk_3549,walk_3550,walk_3551
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dataY.head()

,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,...,class_530,class_531,class_532,class_533,class_534,class_535,class_536,class_537,class_538,class_539
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 3551
Output Classes: 539


In [8]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu'))
deepModel.add(Dense(1024, activation='relu'))
deepModel.add(Dense(768, activation='relu'))
deepModel.add(Dense(512, activation='relu'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid'))

models = [logisticRegression, simpleModel, deepModel]

In [16]:
# Compile model
from keras import losses
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss=losses.binary_crossentropy, optimizer='adam', metrics=[f1score])

In [20]:
def fit_data(model, dataX, dataY):
    # Fit the model
    model.fit(dataX.as_matrix(), dataY.as_matrix(), epochs=1, verbose=2, batch_size=256)

In [21]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives
#     for class_i in range(0,y_true.shape[0]):
#         for class_j in range(0,y_true.shape[1]):
#             y_true_class_i = y_true[class_i,class_j]
#             y_pred_class_i = y_pred[class_i,class_j]
#             if(y_true_class_i>0.5):
#                 if(y_pred_class_i>0.5):
#                     confusionMatrix[1][1]+=1
#                 else:
#                     confusionMatrix[1][0]+=1
#             else:
#                 if(y_pred_class_i>=0.5):
#                     confusionMatrix[0][1]+=1
#                 else:
#                     confusionMatrix[0][0]+=1
#     return confusionMatrix

In [ ]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEpochs = 15
for epoch in range(0, numberOfEpochs):
    print('\n***********************************\nMain Epoch {}/{}'.format(epoch+1, numberOfEpochs))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        print('Training...')
        for model in models:
            print('Model = {}'.format(model.name))
            fit_data(model,dataX, dataY)
            #print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        print('==========================================\n')
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
        
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[trainIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX.as_matrix())
            true_positives, predicted_positives, possible_positives = countPredictions(dataY.as_matrix(), predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print('Testing score: ')
        print('   Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')